In [66]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, log_loss, confusion_matrix, precision_score, recall_score
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import binarize
import math


In [67]:
df = pd.read_csv("Edmonton_Area_Air_Quality_Readings__2017.csv")
df.head(5)

,Row ID,Monitoring Station Name,Parameter Measured,Average Daily Value,Unit of Measure,Date Measured,Latitude,Longitude,Geo Location
0,Power-2017-01-01-Nitric Oxide-0.0000,Power,Nitric Oxide,0.0000,ppm,"Jan 01, 2017",53.633092,-114.420358,POINT (-114.420358 53.633092)
1,Power-2017-01-01-Nitrogen Dioxide-0.0001,Power,Nitrogen Dioxide,0.0001,ppm,"Jan 01, 2017",53.633092,-114.420358,POINT (-114.420358 53.633092)
2,Power-2017-01-01-Outdoor Air Temperature--11.5167,Power,Outdoor Air Temperature,-11.5167,deg c,"Jan 01, 2017",53.633092,-114.420358,POINT (-114.420358 53.633092)
3,Power-2017-01-01-PM2.5 Mass-2.1667,Power,PM2.5 Mass,2.1667,ug/m3,"Jan 01, 2017",53.633092,-114.420358,POINT (-114.420358 53.633092)
4,Power-2017-01-01-Relative Humidity-79.3750,Power,Relative Humidity,79.3750,percent,"Jan 01, 2017",53.633092,-114.420358,POINT (-114.420358 53.633092)


In [68]:
parameters = df['Parameter Measured'].value_counts()
parameters

Outdoor Air Temperature            8685
Wind Direction                     8629
Wind Speed                         8602
Nitric Oxide                       7803
Nitrogen Dioxide                   7803
Total Oxides Of Nitrogen           7803
Sulphur Dioxide                    7208
Relative Humidity                  6640
PM2.5 Mass                         6376
Ozone                              5408
Std. Dev. of Wind Direction        5239
Air Quality Index                  5110
Total Hydrocarbons                 3214
Hydrogen Sulphide                  2875
Methane                            2499
Non-methane Hydrocarbons           2499
Std. Dev. of Wind Speed            2491
Carbon Monoxide                    1812
Solar Radiation                     727
Delta Temperature                   723
Ammonia                             707
Photosynthetic Active Radiation     703
Ethylene                            670
Catch of Standard Collector         365
Outdoor Air Temperature at 10m      359


In [69]:
df.rename(columns = {'Date Measured': 'Date/Time'}, inplace = True)
df['Date/Time'] = df['Date/Time'].astype('datetime64[ns]')

In [70]:
df_edmonton = df[df['Monitoring Station Name'].isin({'Edmonton Central','Edmonton South', 'Edmonton East'})]
df_edmonton.sample(3)

,Row ID,Monitoring Station Name,Parameter Measured,Average Daily Value,Unit of Measure,Date/Time,Latitude,Longitude,Geo Location
74401,Edmonton South-2017-09-12-PM2.5 Mass-2.7500,Edmonton South,PM2.5 Mass,2.7500,ug/m3,2017-09-12,53.500139,-113.526056,POINT (-113.526056 53.500139)
96771,Edmonton East-2017-11-27-Total Hydrocarbons-2....,Edmonton East,Total Hydrocarbons,2.1667,ppm,2017-11-27,53.548139,-113.368194,POINT (-113.368194 53.548139)
16424,Edmonton South-2017-02-26-Ozone-0.0377,Edmonton South,Ozone,0.0377,ppm,2017-02-26,53.500139,-113.526056,POINT (-113.526056 53.500139)


In [71]:
df_carbon = df_edmonton[df_edmonton['Parameter Measured'] == 'Carbon Monoxide']
df_carbon.head()

,Row ID,Monitoring Station Name,Parameter Measured,Average Daily Value,Unit of Measure,Date/Time,Latitude,Longitude,Geo Location
161,Edmonton Central-2017-01-01-Carbon Monoxide-0....,Edmonton Central,Carbon Monoxide,0.2333,ppm,2017-01-01,53.544453,-113.498842,POINT (-113.498842 53.544453)
171,Edmonton East-2017-01-01-Carbon Monoxide-0.2208,Edmonton East,Carbon Monoxide,0.2208,ppm,2017-01-01,53.548139,-113.368194,POINT (-113.368194 53.548139)
188,Edmonton South-2017-01-01-Carbon Monoxide-0.3250,Edmonton South,Carbon Monoxide,0.3250,ppm,2017-01-01,53.500139,-113.526056,POINT (-113.526056 53.500139)
453,Edmonton Central-2017-01-02-Carbon Monoxide-0....,Edmonton Central,Carbon Monoxide,0.3261,ppm,2017-01-02,53.544453,-113.498842,POINT (-113.498842 53.544453)
463,Edmonton East-2017-01-02-Carbon Monoxide-0.3333,Edmonton East,Carbon Monoxide,0.3333,ppm,2017-01-02,53.548139,-113.368194,POINT (-113.368194 53.548139)


In [72]:
import plotly.express as px
fig1 = px.line(df_carbon, x='Date/Time', y= 'Average Daily Value', color = 'Monitoring Station Name')
fig1.show()

In [73]:
df_weather = pd.read_csv("Weather_data_2017.csv")
df_weather["Date/Time"] = df_weather["Date/Time"].astype('datetime64[ns]')
df_weather.sample(3)

,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Data Quality,Max Temp (°C),...,Total Snow (cm),Total Snow Flag,Total Precip (mm),Total Precip Flag,Snow on Grnd (cm),Snow on Grnd Flag,Dir of Max Gust (10s deg),Dir of Max Gust Flag,Spd of Max Gust (km/h),Spd of Max Gust Flag
296,-113.52,53.57,EDMONTON BLATCHFORD,3012209,2017-10-24,2017,10,24,NaN,18.5,...,NaN,M,0.0,NaN,NaN,NaN,32.0,NaN,52,NaN
326,-113.52,53.57,EDMONTON BLATCHFORD,3012209,2017-11-23,2017,11,23,NaN,4.9,...,NaN,M,1.5,NaN,12.0,NaN,32.0,NaN,35,NaN
229,-113.52,53.57,EDMONTON BLATCHFORD,3012209,2017-08-18,2017,8,18,NaN,26.3,...,NaN,M,3.1,NaN,NaN,NaN,32.0,NaN,67,NaN


In [74]:
df_aqweather = pd.merge(df_weather, df_carbon, on = 'Date/Time', how = 'inner')
df_aqweather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1085 entries, 0 to 1084
Data columns (total 39 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Longitude (x)              1085 non-null   float64       
 1   Latitude (y)               1085 non-null   float64       
 2   Station Name               1085 non-null   object        
 3   Climate ID                 1085 non-null   int64         
 4   Date/Time                  1085 non-null   datetime64[ns]
 5   Year                       1085 non-null   int64         
 6   Month                      1085 non-null   int64         
 7   Day                        1085 non-null   int64         
 8   Data Quality               0 non-null      float64       
 9   Max Temp (°C)              1079 non-null   float64       
 10  Max Temp Flag              9 non-null      object        
 11  Min Temp (°C)              1085 non-null   float64       
 12  Min Te

In [75]:
#df2_edmonton = df_aqweather.groupby('Monitoring Station Name')
#df2_edmonton = pd.DataFrame(df2_edmonton)
#df2_edmonton.head()

In [76]:
fig2 = px.bar(df_aqweather, x='Date/Time', y= 'Mean Temp (°C)', color = 'Monitoring Station Name')
fig2.show()